In [6]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [2]:
import tensorflow

(x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.mnist.load_data()
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.uint8)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.uint8)

2025-01-22 07:09:28.778886: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-22 07:09:28.880290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737509968.918919    6301 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737509968.930178    6301 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-22 07:09:29.023216: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [18]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.ff1 = nn.Linear(784, 100)
        self.ff2 = nn.Linear(100, 100)
        self.ff3 = nn.Linear(100, 100)
        self.output = nn.Linear(100, 10)
        self.activation = nn.ReLU()
        self.loss = nn.CrossEntropyLoss()
        self.trainloader = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(x_train, y_train),
            batch_size=1000,
            shuffle=True,
        )
        self.validloader = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(x_test, y_test),
            batch_size=1000,
            shuffle=True,
        )
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        self.lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode="min", factor=0.5, patience=15, verbose=True
        )

    def forward(self, x):
        x = self.ff1(x)
        x = self.activation(x)
        x = self.ff2(x)
        x = self.activation(x)
        x = self.ff3(x)
        x = self.activation(x)
        x = self.output(x)
        return x

    def train_loop(self, run_name, epochs=100):
        previous_lr = self.optimizer.param_groups[0]["lr"]
        writer = SummaryWriter(log_dir=f"runs/{run_name}")
        for i in range(epochs):
            t = 0
            self.train()
            train_progress = tqdm(
                enumerate(self.trainloader),
                desc=f"Epoch {i} - Training",
                total=len(self.trainloader),
            )
            for j, k in train_progress:
                image, label = k
                image, label = image.to("cuda"), label.to("cuda")
                self.optimizer.zero_grad()
                output = self(image.view(-1, 784))
                l = self.loss(output, label)
                l.backward()
                self.optimizer.step()
                t += l.item()
                train_progress.set_postfix(loss=t / (j + 1))
            t /= len(self.trainloader)
            val_loss = self.val_loop(i)
            self.lr_scheduler.step(val_loss)
            writer.add_scalars("Losses", {"Train": t, "Validation": val_loss}, i)
            writer.flush()
            current_lr = self.optimizer.param_groups[0]["lr"]
            if current_lr != previous_lr:
                print(f"Learning Rate changed to {current_lr}")
                previous_lr = current_lr
        writer.close()

    def val_loop(self, i):
        t = 0
        self.eval()
        val_progress = tqdm(
            enumerate(self.validloader),
            desc=f"Epoch {i} - Validation",
            total=len(self.validloader),
        )
        correct = 0
        for j, k in val_progress:
            image, label = k
            image, label = image.to("cuda"), label.to("cuda")
            output = self(image.view(-1, 784))
            l = self.loss(output, label)
            pred = output.softmax(dim=1).argmax(dim=1)
            correct += (pred == label).sum().item()
            t += l.item()
            val_progress.set_postfix(loss=t / (j + 1))
            val_progress.set_postfix(accuracy=correct / ((j + 1) * 1000))
        t /= len(self.validloader)
        return t

In [19]:
model = Model().to("cuda")
model.train_loop("No-Regularization")

/home/shusrith/projects/torch/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 22 - Validation: 100%|██████████| 10/10 [00:00<00:00, 133.19it/s, accuracy=0.974]


Learning Rate changed to 0.0005


Epoch 38 - Validation: 100%|██████████| 10/10 [00:00<00:00, 134.50it/s, accuracy=0.978]


Learning Rate changed to 0.00025


Epoch 54 - Validation: 100%|██████████| 10/10 [00:00<00:00, 122.52it/s, accuracy=0.979]


...

Epoch 70 - Validation: 100%|██████████| 10/10 [00:00<00:00, 184.78it/s, accuracy=0.979]


Learning Rate changed to 6.25e-05


Epoch 86 - Validation: 100%|██████████| 10/10 [00:00<00:00, 172.56it/s, accuracy=0.979]


Learning Rate changed to 3.125e-05


Epoch 99 - Validation: 100%|██████████| 10/10 [00:00<00:00, 52.71it/s, accuracy=0.979]
